# LineMOD データセット読み出し

In [3]:
import os
import sys
sys.path.append(".")
sys.path.append("..")

In [4]:
linemod_objects = [
    "ape",
    "benchviseblue",
    "bowl",
    "cam",
    "can",
    "cat",
    "cup",
    "driller",
    "duck",
    "eggbox",
    "glue",
    "holepuncher",
    "iron",
    "lamp",
    "phone"
]

## パス設定

In [5]:
from config.config import cfg

base_dir = cfg.LINEMOD_DIR

In [6]:
object_name = 'ape'

if object_name == 'all':
    object_names = linemod_objects
elif object_name in linemod_objects:
    object_names = [object_name]
else:
    raise ValueError('Invalid object name: {}'.format(object_name))

## その他の設定

In [7]:
img_shape = (480, 960) # (h, w)
lengths = {}
total_length = 0
for object_name in object_names:
    length = len(list(filter(lambda x:x.endswith('jpg'),
                                                os.listdir(os.path.join(
                                                    base_dir,
                                                    object_name,
                                                    'data')))))
    lengths[object_name] = length
    total_length += length

print(total_length)

1236


|object name| number of datas |
|---|---|
| ape | 1236 |
| benchviseblue | 1215 |
| bowl | 1233 |
| cam | 1201 |
| can | 1196 |
| cat | 1179 |
| cup | 1240 |
| driller | 1188 |
| duck | 1254 |
| eggbox | 1253 |
| glue | 1220 |
| holepuncher | 1237 |
| iron | 1152 |
| lamp | 1227 |
| phone | 1243 |
| all | 18274 |

## 3d point の読み込み

In [8]:
from glob import glob
import numpy as np

In [9]:
filenames = glob(os.path.join(base_dir, object_name, '*.ply'))

print(filenames)

['/media/miki/DATA/My_programing/python/AI/Analysis/data/linemod/ape/mesh.ply', '/media/miki/DATA/My_programing/python/AI/Analysis/data/linemod/ape/OLDmesh.ply']


In [10]:
def read_3d_points(filename):
        with open(filename) as f:
            in_vertex_list = False
            vertices = []
            in_mm = False
            for line in f:
                if in_vertex_list:
                    vertex = line.split()[:3]
                    vertex = np.array([[float(vertex[0])],
                                                          [float(vertex[1])],
                                                          [float(vertex[2])]],
                                                          dtype=np.float32)
                    if in_mm:
                        vertex = vertex / np.float32(10) # mm -> cm
                    vertex = vertex / np.float32(100) # cm -> m
                    vertices.append(vertex)
                    if len(vertices) >= vertex_count:
                        break
                elif line.startswith('element vertex'):
                    vertex_count = int(line.split()[-1])
                elif line.startswith('end_header'):
                    in_vertex_list = True
                elif line.startswith('element face'):
                    in_mm = True
        return vertices

In [11]:
filename = filenames[0]

vertices = read_3d_points(filename)

print(vertices)


       [-0.0159688]], dtype=float32), array([[-0.0216407],
       [-0.0146209],
       [-0.0164644]], dtype=float32), array([[ 0.00045176],
       [ 0.0395507 ],
       [-0.0410252 ]], dtype=float32), array([[ 0.0188079 ],
       [ 0.0189133 ],
       [-0.06827381]], dtype=float32), array([[-0.0333898 ],
       [-0.0073597 ],
       [-0.00125748]], dtype=float32), array([[-0.0058811 ],
       [-0.00097199],
       [-0.0500007 ]], dtype=float32), array([[ 0.0157192],
       [ 0.0133282],
       [-0.0861312]], dtype=float32), array([[ 0.0169909],
       [ 0.0119787],
       [-0.0857204]], dtype=float32), array([[ 0.0387356],
       [-0.0113963],
       [-0.018281 ]], dtype=float32), array([[ 0.0381139],
       [-0.0125783],
       [-0.0193455]], dtype=float32), array([[ 0.039341 ],
       [ 0.0012703],
       [-0.0363044]], dtype=float32), array([[ 0.0129079 ],
       [-0.021099  ],
       [-0.00146649]], dtype=float32), array([[-0.0173083],
       [-0.0217725],
       [-0.0437112]], dt

## Rotation

In [12]:
def read_rotation(filename):
     """LineMod3Dモデルの回転行列を計算する関数

        Arg:
            filename (str): 単一のLineMod3Dモデルの回転情報が保存された '.rot' ファイルへの絶対パス

        Return:
            R [np.matrix]: 単一のLineMod3Dモデルの(3, 3)回転行列
        """
        with open(filename) as f:
            f.readline()
            R = []
            for line in f:
                R.append(line.split())
            R = np.array(R, dtype=np.float32)
        return R

In [13]:
filenames = glob(os.path.join(base_dir, object_names[0], 'data', '*.rot'))

R = read_rotation(filenames[0])

print(R)

[[-0.63252    0.773706  -0.0360155]
 [-0.358213  -0.250986   0.899272 ]
 [ 0.686733   0.581709   0.435905 ]]


## Translation

In [16]:
def read_translation(filename: str) -> np.matrix:
        """LineMod3Dモデルの並進行列を計算する関数

        Arg:
            filename (str): 単一のLineMod3Dモデルの並進情報が保存された '.tra' ファイルへの絶対パス

        Return:
            T [np.matrix]: 単一のLineMod3Dモデルの(3, 3)並進行列
        """
        with open(filename) as f:
            f.readline()
            T = []
            for line in f:
                T.append([line.split()[0]])
            T = np.array(T, dtype=np.float32)
            T = T / np.float32(100) # cm -> m
        return T

In [17]:
filenames = glob(os.path.join(base_dir, object_names[0], 'data', '*.tra'))

T = read_translation(filenames[0])

print(T)

[[-0.0522575 ]
 [-0.0308449 ]
 [ 0.79460406]]
